In [18]:
# https://pypi.org/project/exif/
# pip install exif

# https://pypi.org/project/pymediainfo/
# pip install pymediainfo

In [104]:
import pandas as pd
# import PIL.Image
from exif import Image
from pprint import pprint
from pymediainfo import MediaInfo
from pathlib import Path
import numpy as np
import os
import re
import numpy as np

In [49]:
path = Path(r'G:\Google Photo_extr\Takeout\Google Фото')
# path = Path(r'D:\Рабочая\!_Scripts\change_file_creation_time')
# path = Path('G:/temp/Тест фото')

In [50]:
%%time
df = pd.DataFrame(columns=['name', 'full_name', 'extension', 'path'])
for file_path in path.glob('**/*.*'):
    if file_path.is_file():
        extension = file_path.suffix.lower()
        name = Path(file_path.stem).stem
        full_name = file_path.name
        parent_path = file_path.parent
        df.loc[len(df)] = [name, full_name, extension, parent_path]
df.head()

Wall time: 6min 14s


,name,full_name,extension,path
0,print-subscriptions,print-subscriptions.json,.json,G:\Google Photo_extr\Takeout\Google Фото
1,shared_album_comments,shared_album_comments.json,.json,G:\Google Photo_extr\Takeout\Google Фото
2,user-generated-memory-titles,user-generated-memory-titles.json,.json,G:\Google Photo_extr\Takeout\Google Фото
3,DSC01856,DSC01856.JPG,.jpg,G:\Google Photo_extr\Takeout\Google Фото\Photo...
4,DSC01856,DSC01856.JPG.json,.json,G:\Google Photo_extr\Takeout\Google Фото\Photo...


In [102]:
df['full_path'] = df.path / df.full_name

In [103]:
df.full_path.loc[0]

WindowsPath('G:/Google Photo_extr/Takeout/Google Фото/print-subscriptions.json')

In [99]:
df.full_name.loc[0]

'print-subscriptions.json'

In [89]:
# посмотрим сколько каких файлов по типам расширения у нас есть
df.extension.value_counts()

.json    31740
.jpg     28857
.mp4      2132
.mts       516
.jpeg       56
.avi        41
.nef        40
.mov        38
.3gp        19
.gif        13
.png         6
.mpg         3
.bmp         3
.mkv         1
Name: extension, dtype: int64

In [52]:
# списки расширений для видео и фото
video_ext = np.array(['.mp4', '.mts', '.avi', '.mov', '.3gp', '.mpg', '.mkv'])
photo_ext = np.array(['.jpg', '.jpeg', '.nef', '.gif', '.png', '.bmp'])

In [83]:
nef_path = df.query('extension == ".nef"').path

In [86]:
nef_path.loc[15967]

WindowsPath('G:/Google Photo_extr/Takeout/Google Фото/Photos from 2012')

In [60]:
# фильтруем медиа-файлы
media_df = df.query('extension in @photo_ext or extension in @video_ext')

In [61]:
# фильтруем json-файлы
json_df = df.query('extension == ".json"')

In [62]:
media_df[['name', 'full_name']].shape

(31725, 2)

In [72]:
# проверяем для каждого ли медиа файла имеется свой json-файл?
comparison = (media_df[['name', 'full_name', 'path']]
    .merge(json_df[['name', 'full_name']]
           , on='name'
           , how = 'left')
             )

In [73]:
comparison.isna().sum()

name              0
full_name_x       0
path              0
full_name_y    3351
dtype: int64

In [82]:
# медиа файлы без json-ов
comparison[comparison.full_name_y.isna()]

,name,full_name_x,path,full_name_y
84,DSC01887-измененный,DSC01887-измененный.JPG,G:\Google Photo_extr\Takeout\Google Фото\Photo...,NaN
95,DSC01893-измененный,DSC01893-измененный.JPG,G:\Google Photo_extr\Takeout\Google Фото\Photo...,NaN
111,DSC01911-измененный,DSC01911-измененный.JPG,G:\Google Photo_extr\Takeout\Google Фото\Photo...,NaN
3289,scene_20070728 17.58,scene_20070728 17.58.48.mp4,G:\Google Photo_extr\Takeout\Google Фото\Photo...,NaN
3290,scene_20070728 18.07,scene_20070728 18.07.14.mp4,G:\Google Photo_extr\Takeout\Google Фото\Photo...,NaN
...,...,...,...,...
127197,DSC_0218(1),DSC_0218(1).JPG,G:\Google Photo_extr\Takeout\Google Фото\Савел...,NaN
127211,DSC_0219(1),DSC_0219(1).JPG,G:\Google Photo_extr\Takeout\Google Фото\Савел...,NaN
127667,DSC_0257(1),DSC_0257(1).JPG,G:\Google Photo_extr\Takeout\Google Фото\Савел...,NaN
127685,DSC_0258(1),DSC_0258(1).JPG,G:\Google Photo_extr\Takeout\Google Фото\Савел...,NaN


In [110]:
# функция вытаскивающая exif
def get_exif(path_col):
    
    # списки расширений для видео и фото
    video_ext = np.array(['.mp4', '.mts', '.avi', '.mov', '.3gp', '.mpg', '.mkv'])
    photo_ext = np.array(['.jpg', '.jpeg', '.nef', '.gif', '.png', '.bmp'])
    
    # вытаскиваем exif из видео
    if path_col.suffix.lower() in video_ext:
        try:
            media_info = MediaInfo.parse(path_col)
            for track in media_info.tracks:
                if track.track_type == "Video":
                    exif_vdata = track.to_data()
            vdata = exif_vdata['tagged_date']
            vdata = pd.to_datetime(vdata[4:])
            time_zone = pd.to_timedelta(7, unit='h')
            return pd.to_datetime(vdata[4:]) + time_zone
        except:
            return np.nan()
    
    # вытаскиваем exif из фото
    elif path_col.suffix.lower() in photo_ext :
        try:
            with open('IMG_20211123_162603.jpg', 'rb') as image_file:
                my_image = Image(image_file)
            fdata = my_image.datetime
            fpattern = re.compile('(\d\d\d\d:\d\d:\d\d) (\d\d:\d\d:\d\d)') 
            return pd.to_datetime(fpattern.findall(fdata)[0][0].replace(':','-') + ' ' + fpattern.findall(fdata)[0][1])
        except:
            return np.nan
    else:
        return np.nan

In [107]:
df.full_path.loc[3]

WindowsPath('G:/Google Photo_extr/Takeout/Google Фото/Photos from 2006/DSC01856.JPG')

In [111]:
get_exif(df.full_path.loc[3])

nan

In [46]:
media_info = MediaInfo.parse("VID_20190106_162804.mp4")
for track in media_info.tracks:
    if track.track_type == "Video":
        exif_vdata = track.to_data()
vdata = exif_vdata['tagged_date']

'UTC 2019-01-07 00:28:08'

In [50]:
pd.to_datetime(vdata[4:])

Timestamp('2019-01-07 00:28:08')

In [55]:
time_zone = pd.to_timedelta(7, unit='h')

In [56]:
pd.to_datetime(vdata[4:]) + time_zone

Timestamp('2019-01-07 07:28:08')

In [59]:
with open('IMG_20211123_162603.jpg', 'rb') as image_file:
    my_image = Image(image_file)
my_image.has_exif

True

In [72]:
# список всех методов
# my_image.list_all()

In [63]:
fdata = my_image.datetime
fdata

'2021:11:23 16:26:05'

In [71]:
fpattern = re.compile('(\d\d\d\d:\d\d:\d\d) (\d\d:\d\d:\d\d)') 
fpattern.findall(fdata)[0][0].replace(':','-') + ' ' + fpattern.findall(fdata)[0][1]

'2021-11-23 16:26:05'

In [25]:
my_image.datetime_original

'2021:11:23 16:26:05'

In [26]:
my_image.datetime_digitized

'2021:11:23 16:26:05'

In [27]:
my_image.gps_datestamp

'2021:11:23'